# This demo is deprecated and only works with older versions of `qlbm`.

### Runtime Comparison

This notebook showcases how `qlbm` enables the comparison of multiple runners for the analysis of runtime performance of QBMs.

We will use the built-in logger of `qlbm` to extract performance data and analyse which of Qiskit and Qulacs perform better on a set of benchmarks.

In [ ]:
%pip install qlbm matplotlib seaborn pandas

In [ ]:
# Import the required runtime utilities
import logging.config
from logging import Logger, getLogger

from qiskit_aer import AerSimulator

from qlbm.components import (
    CQLBM,
    CollisionlessInitialConditions,
    EmptyPrimitive,
    GridMeasurement,
)
from qlbm.infra import QiskitRunner, QulacsRunner, SimulationConfig
from qlbm.lattice import CollisionlessLattice
from qlbm.tools.utils import get_circuit_properties

In [ ]:
# Import analysis and plotting utilities
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme()

## Step 1: Setup

Before benchmarking the simulations, we first have to choose some behaviour to simulate and parameters for our quantum circuits.

In [ ]:
def benchmark(
    lattice_dicts,
    num_steps: int,
    num_shots: int,
    statevector_snapshots: bool,
    statevector_sampling: bool,
    logger: Logger,
    dummy_logger: Logger,
) -> None:
    for count, lattice_dict in enumerate(lattice_dicts):
        logger.info(f"Combination #{count + 1} of {len(lattice_dicts)}")

        lattice = CollisionlessLattice(lattice_dict, logger=dummy_logger)
        logger.info(
            f"Lattice={lattice.logger_name()}, num_qubits={lattice.num_total_qubits}"
        )

        ####################
        # Running Qiskit
        ####################

        logger.info("Executing QISKIT...")
        output_dir = f"qlbm-output/qiskit-qulacs-comparison/qiskit-{lattice.logger_name()}-{statevector_snapshots}-{statevector_snapshots}"

        cfg = SimulationConfig(
            initial_conditions=CollisionlessInitialConditions(lattice, dummy_logger),
            algorithm=CQLBM(lattice, dummy_logger),
            postprocessing=EmptyPrimitive(lattice, dummy_logger),
            measurement=GridMeasurement(lattice, dummy_logger),
            target_platform="QISKIT",
            compiler_platform="QISKIT",
            optimization_level=0,
            statevector_sampling=statevector_sampling,
            execution_backend=AerSimulator(method="statevector"),
            sampling_backend=AerSimulator(method="statevector"),
            logger=dummy_logger,
        )

        cfg.prepare_for_simulation()
        logger.info(
            f"Final circuit properties: {get_circuit_properties(cfg.algorithm)}"
        )

        # Create a runner object to simulate the circuit
        runner = QiskitRunner(cfg, lattice, logger=logger)

        # Simulate the circuits using both snapshots and sampling
        runner.run(
            num_steps,  # Number of time steps
            num_shots,  # Number of shots per time step
            output_dir,
            statevector_snapshots=statevector_snapshots,
        )

        ####################
        # Running Qulacs
        ####################

        logger.info("Executing QULACS...")
        output_dir = f"qlbm-output/qiskit-qulacs-comparison/qulacs-{lattice.logger_name()}-{statevector_snapshots}-{statevector_snapshots}"

        cfg.prepare_for_simulation()
        logger.info(
            f"Final circuit properties: {get_circuit_properties(cfg.algorithm)}"
        )
        cfg = SimulationConfig(
            initial_conditions=CollisionlessInitialConditions(lattice, dummy_logger),
            algorithm=CQLBM(lattice, dummy_logger),
            postprocessing=EmptyPrimitive(lattice, dummy_logger),
            measurement=GridMeasurement(lattice, dummy_logger),
            target_platform="QULACS",
            compiler_platform="TKET",
            optimization_level=0,
            statevector_sampling=statevector_sampling,
            execution_backend=None,
            sampling_backend=AerSimulator(method="statevector"),
            logger=dummy_logger,
        )
        cfg.prepare_for_simulation()
        # Create a runner object to simulate the circuit
        runner = QulacsRunner(cfg, lattice, logger=logger)

        # Simulate the circuits using both snapshots and sampling
        runner.run(
            num_steps,  # Number of time steps
            num_shots,  # Number of shots per time step
            output_dir,
            statevector_snapshots=statevector_snapshots,
        )

In [ ]:
!mkdir -p qlbm-output/qiskit-qulacs-comparison/ && touch qlbm-output/qiskit-qulacs-comparison/qlbm.log
!:> qlbm-output/qiskit-qulacs-comparison/qlbm.log

In [ ]:
# Define some parameters to experiment with
NUM_SHOTS = 2**13
NUM_STEPS = 3

lattices = [
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [],
    },
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [{"x": [5, 6], "y": [1, 2], "boundary": "specular"}],
    },
    {
        "lattice": {"dim": {"x": 8, "y": 8}, "velocities": {"x": 4, "y": 4}},
        "geometry": [
            {"x": [5, 6], "y": [1, 2], "boundary": "specular"},
            {"x": [5, 6], "y": [5, 6], "boundary": "specular"},
        ],
    },
]

dummy_logger = getLogger("dummy")
# By logging at this point we ignore the output of circuit creation
logging.config.fileConfig("qiskit_qulacs_comparison_logging.conf")
logger = getLogger("qlbm")

## Step 2: Simulation

Now that we have setup our simulations and configered logging, we can simply run the simulations by calling the `benchmark` function!

> **_CAUTION:_** Running the cell below will probably take a few minutes. Each cell should be run exactly once.

In [ ]:
logger.info("Session: snapshots=True, sampling=True")
benchmark(lattices, NUM_STEPS, NUM_SHOTS, True, True, logger, dummy_logger)

## Step 3: Analysis

Once the simulation has concluded, the performance logs created by `qlbm` will be under `qlbm-output/qiskit-qulacs-comparison/qlbm.log`.
The scripts below will parse this file, extract useful information, format it, and plot it for convenient analysis.

In [ ]:
log_file = "qlbm-output/qiskit-qulacs-comparison/qlbm.log"
with open(log_file, "r") as f:
    lines = f.readlines()

In [ ]:
combination_lines_indices = [
    c for c, line in enumerate(lines) if "Combination #" in line
]

sections = []
for c in range(len((combination_lines_indices))):
    if c < len(combination_lines_indices) - 1:
        sections.append(
            lines[combination_lines_indices[c] : combination_lines_indices[c + 1]]
        )
    else:
        sections.append(lines[combination_lines_indices[c] :])

records = []
for section in sections:
    sec_info = section[1].split("INFO: ")[-1].rstrip().split(", ")
    time_elapsed_ns = section[-1].split("INFO: ")[-1].rstrip().split()[-2]
    step_simulation_line_indices = [
        c for c, line in enumerate(section) if "Main circuit for step" in line
    ]
    qulacs_start = [c for c, line in enumerate(section) if "Executing QULACS..." in line][
        0
    ]
    total_duration = 0
    for c, sl in enumerate(step_simulation_line_indices):
        if sl > qulacs_start and step_simulation_line_indices[c - 1] < qulacs_start:
            total_duration = 0
        step_number = section[sl].split("for step ")[-1].split()[0]
        props = section[sl].split("INFO: ")[-1].rstrip().split(", ")[1:]
        duration = section[sl + 1].split()[-2]
        total_duration += int(duration)

        records.append(
            {
                "Lattice": sec_info[0].split("=")[-1].split("/")[-1].split(".")[0],
                "Dimensions": sec_info[0].split("=")[-1].split("-")[1],
                "Obstacles": int(sec_info[0].split("=")[-1].split("-")[-2]),
                "Circuit Qubits": int(sec_info[1].split("=")[-1]),
                "Step": int(step_number),
                "Depth": int(props[1]),
                "Gates": int(props[-1][:-1]),
                "Duration (ns)": int(duration),
                "Cumulative Duration (ns)": int(total_duration),
                "Snapshots": True,
                "Platform": "QULACS"
                if sl > qulacs_start
                else "QISKIT",  # Qiskit is first in this simulation
            }
        )

df = pd.DataFrame.from_records(records)
df

In [ ]:
df["Duration (s)"] = df["Duration (ns)"] / 1e9
df["Cumulative Duration (s)"] = df["Cumulative Duration (ns)"] / 1e9

In [ ]:
pdf = df[df["Obstacles"] < 2]
sns.lineplot(
    pdf,
    x="Step",
    y="Cumulative Duration (s)",
    hue="Obstacles",
    style="Platform",
    markers=True,
)

plt.xticks(pd.unique(pdf["Step"]))